In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy.stats import chi2_contingency,chi2
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as pyo
pyo.init_notebook_mode()
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('/kaggle/input/pakistans-largest-ecommerce-dataset/Pakistan Largest Ecommerce Dataset.csv')
print(data.shape) # shape of data
print(data.columns) # list of columns
data

# Subset of data needed for Payment Methods versus Order Status analysis

In [ ]:
data_sub = data[['payment_method', 'status', 'grand_total', 'Year', 'Month']]
data_sub

# Treatment of Null or NaN

In [ ]:
# there are some rows in which all columns are null need to drop all those rows
data_sub.dropna(how='all', inplace=True)

#null_status_rows = pd.isnull(data_sub.status)
data_sub.sort_index(ascending=False)
print(data_sub.shape)

# Status null treatment
> After removing all rows having nan in all column, only 15 are rows in which status is nullwe use mode to all 15 nulls. Mode value is completed.

In [ ]:
data_sub.status.fillna(data_sub.status.mode()[0], inplace=True)
print(data_sub.isnull().sum())

# Updating some payment type methods
> Replacing cashatdoorstep with cod and Easypay_MA with Easypay

In [ ]:
data_sub['payment_method'].replace({'cashatdoorstep': 'cod', 'Easypay_MA': 'Easypay'}, inplace=True)

# Types of Payment Methods And List of statuses

In [ ]:
type_of_payment_methods = data_sub.payment_method.unique()
print(type_of_payment_methods)
print(f'Total number of Payment Method {len(type_of_payment_methods)}')
list_of_statues = data_sub.status.unique()
print(list_of_statues)

# Crosstab/Contingency-Table  of *payment_method* and *status*

In [ ]:
crosstab = pd.crosstab(data_sub.payment_method,data_sub.status)
crosstab

In [ ]:
fig = px.line(crosstab, x=crosstab.index.values, y=crosstab.columns.values)
fig.show()

# Chi-Squared Test
> We can interpret the test statistic in the context of the chi-squared distribution with the requisite number of degress of freedom as follows:
* If Statistic >= Critical Value: significant result, reject null hypothesis (H0), dependent.
* If Statistic < Critical Value: not significant result, fail to reject null hypothesis (H0), independent.

In [ ]:
stat, p, dof, expected = chi2_contingency(crosstab)

# interpret test-statistic
prob = 0.95
critical = chi2.ppf(prob, dof)
if abs(stat) >= critical:
    print('Dependent (reject H0)')
else:
    print('Independent (fail to reject H0)')

> In terms of a p-value and a chosen significance level (alpha), the test can be interpreted as follows:
* If p-value <= alpha: significant result, reject null hypothesis (H0), dependent.
* If p-value > alpha: not significant result, fail to reject null hypothesis (H0), independent.

In [ ]:
# interpret p-value
alpha = 1.0 - prob
if p <= alpha:
    print('Dependent (reject H0)')
else:
    print('Independent (fail to reject H0)')

# Conclusion
![Conclusion](https://islamphobia.files.wordpress.com/2010/12/conclusion-isolated-on-special-blue-260nw-1151283452-e1554147806139.jpg)
> After Contingency table and Chi-squared Tests, we can conclude that payment method and status of orders are dependent on each other.